In [7]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn import preprocessing

housing_data = datasets.fetch_california_housing(data_home='./')

### Preprocessing

housing_data_scaled = preprocessing.scale(housing_data.data)
for idx in range(housing_data.data.shape[1]):
    housing_data_scaled[1] = housing_data_scaled.mean(axis=0)
    
### TF

pred = tf.Variable(tf.random_uniform([1], -1.0, 1.0)) # bias 
for idx in range(housing_data.data.shape[1]): # weight * x
    pred  += (  tf.Variable(tf.random_uniform([1], -1.0, 1.0), name='W_'+str(idx)) * housing_data_scaled[:, idx]) 

loss = tf.reduce_mean(tf.square(pred - housing_data.target)) 
train = tf.train.AdamOptimizer(0.001).minimize(loss)
init = tf.global_variables_initializer()

### run Session

with tf.Session() as sess:
    sess.run(init)
    for step in range(10000):
        sess.run(train)
        if step % 100 == 0:
            print(step, sess.run(loss))

0 9.96501
100 8.18464
200 6.77471
300 5.67275
400 4.81596
500 4.14774
600 3.62005
700 3.19482
800 2.84397
900 2.54795
1000 2.29374
1100 2.07265
1200 1.87875
1300 1.70771
1400 1.55629
1500 1.42193
1600 1.3026
1700 1.19664
1800 1.10268
1900 1.01958
2000 0.946352
2100 0.882091
2200 0.825993
2300 0.777296
2400 0.735287
2500 0.699284
2600 0.668644
2700 0.642755
2800 0.621044
2900 0.602971
3000 0.588042
3100 0.575798
3200 0.565826
3300 0.557756
3400 0.551261
3500 0.546056
3600 0.541897
3700 0.538578
3800 0.535928
3900 0.533807
4000 0.5321
4100 0.530721
4200 0.5296
4300 0.528681
4400 0.527925
4500 0.527298
4600 0.526777
4700 0.526344
4800 0.525983
4900 0.525683
5000 0.525434
5100 0.525227
5200 0.525058
5300 0.52492
5400 0.524808
5500 0.524718
5600 0.524647
5700 0.524591
5800 0.524547
5900 0.524514
6000 0.524488
6100 0.524469
6200 0.524456
6300 0.524446
6400 0.524438
6500 0.524434
6600 0.52443
6700 0.524428
6800 0.524427
6900 0.524426
7000 0.524426
7100 0.524425
7200 0.524425
7300 0.524425
740